<a href="https://colab.research.google.com/github/kimberley-faria/implicit-hate-detection/blob/main/implicit_hate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install wandb
# !wandb login

     |████████████████████████████████| 1.7 MB 14.7 MB/s 
     |████████████████████████████████| 181 kB 34.3 MB/s 
     |████████████████████████████████| 144 kB 41.3 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=14855922794df4df3d765f453e5a70d7dd310defa20d4b46599c208f6e97384f
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
# import wandb

# wandb.init(project="test-project", entity="cs685-implicit-hate-detection")

ModuleNotFoundError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Github/685/implicit-hate-detection
!git checkout 1-implicit-hate-dataset-loader
!git status

/content/drive/MyDrive/Github/685/implicit-hate-detection
M	implicit_hate_dataloader/dataloader.py
M	implicit_hate_detection.ipynb
M	requirements.txt
Already on '1-implicit-hate-dataset-loader'
Your branch is up to date with 'origin/1-implicit-hate-dataset-loader'.
On branch 1-implicit-hate-dataset-loader
Your branch is up to date with 'origin/1-implicit-hate-dataset-loader'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   implicit_hate_dataloader/dataloader.py
	modified:   implicit_hate_detection.ipynb
	modified:   requirements.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 3.8 MB 13.0 MB/s 
     |████████████████████████████████| 6.5 MB 39.8 MB/s 
     |████████████████████████████████| 895 kB 42.0 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/Github/685/implicit-hate-detection/')

In [6]:
%load implicit_hate_dataloader/dataloader.py    
%load implicit_hate_dataloader/helpers.py

In [8]:
import torch
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla K80, n_gpu: 1


In [9]:
import os

data_path = '../data/implicit-hate-corpus-nov-2021/implicit-hate-corpus'
dataset_filename = {
    # post (str)
    # class (str): high-level label in {`explicit_hate`,`implicit_hate`,`not_hate`}
    "stage-1": os.path.join(data_path, "implicit_hate_v1_stg1_posts.tsv"),

    # post (str)
    # class (str): fine-grained implicit hate label in
    # {`white_grievance`, `incitement`, `inferiority`, `irony`, `stereotypical`, `threatening`, `other`}
    # extra_implicit_class: (str) A secondary fine-grained implicit hate label in
    # {`white_grievance`, `incitement`, `inferiority`, `irony`, `stereotypical`, `threatening`, `other`, **None**}
    "stage-2": os.path.join(data_path, "implicit_hate_v1_stg2_posts.tsv"),

    # post (str)
    # target: (str) Free-text annotation for the group being targeted (e.g. `Black people`, `Immigrants`, etc.)
    # implied_statement: (str) Free-text annotation for the implicit or hidden underlying meaning of the post made
    # explicit (e.g. `people in minority groups are all in gangs`)
    "stage-3": os.path.join(data_path, "implicit_hate_v1_stg3_posts.tsv")
}

In [10]:
from implicit_hate_dataloader.dataloader import Stage1Dataset, Stage2Dataset
from torch.utils.data import DataLoader

In [11]:
# Stage 1 Annotations: see config.py for dataset specs
training_data = Stage1Dataset(dataset_filename["stage-1"])
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=False)

# Print post and label.
posts, classes, input_ids, attention_masks, labels = next(iter(train_dataloader))
print('Original: ', posts[0], classes[0])
print('Token IDs:', input_ids[0])
print('Attention Mask IDs:', attention_masks[0])
print('Label:', labels[0])

print(len(training_data))

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Original:    " : jewish harvard professor noel ignatiev wants to abolish the white race via #wr " " " implicit_hate
Token IDs: tensor([  101,  1000,  1024,  3644,  5765,  2934, 10716, 16270, 10450,  6777,
         4122,  2000, 11113, 20872,  2232,  1996,  2317,  2679,  3081,  1001,
        23277,  1000,  1000,  1000,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
Attention Mask IDs: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Label: tensor(1)
21480


In [12]:
# Stage 2 Annotations: see config.py for dataset specs
training_data = Stage2Dataset(dataset_filename["stage-2"])
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=False)

# Print post and label.
posts, im_classes, ext_im_classes, input_ids, attention_masks, im_labels, ext_im_labels = next(iter(train_dataloader))
print('Original: ', posts[0], im_classes[0], ext_im_classes[0])
print('Token IDs:', input_ids[0])
print('Attention Mask IDs:', attention_masks[0])
print('Label:', im_labels[0], ext_im_labels[0])

print(len(training_data))  # Coroborated with paper section 4.2.3

Original:    " : jewish harvard professor noel ignatiev wants to abolish the white race via #wr " " " white_grievance None
Token IDs: tensor([  101,  1000,  1024,  3644,  5765,  2934, 10716, 16270, 10450,  6777,
         4122,  2000, 11113, 20872,  2232,  1996,  2317,  2679,  3081,  1001,
        23277,  1000,  1000,  1000,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
Attention Mask IDs: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Label: tensor(0) tensor(7)
6346
